#### I tried tweaking the hard-coded values shown (test_pct, num_feat, trees) but could not get above 64% accuracy. Increasing the number of trees above 500 or so crashes the kernel

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
from operator import itemgetter

TEST_PCT = 0.60
NUM_FEAT = 7
NUM_TREES = 300
COLS = ['density', 'median_age', 'median_hh_income', 'pct_hispanic',
        'pct_white', 'pct_black', 'avg_hh_size']

In [ ]:
# Load the data
if sys.platform[:3] =='win':
    data_loc = 'D:/Data/Loyalty Vision/'
else:
    data_loc = "/home/tom/ML/data/CBG/"

filenm = "CBG_prepared.csv"
df = pd.read_csv(data_loc+filenm, delimiter=',')
df = df[:10000]

In [ ]:
# Split into Train and Test
df_train, df_test = train_test_split(df, test_size=TEST_PCT)

train_x = df_train[COLS]
train_y = df_train['spend_per_hh']
test_x = df_test[COLS]
test_y = df_test['spend_per_hh']
print('Record counts    Training: {:,}   Test  {:,}'.format(df_train.shape[0], df_test.shape[0]))

In [ ]:
# Run the job
start = time.time()

# Fit regression model
regr = RandomForestRegressor(oob_score=True, max_features=NUM_FEAT, n_estimators=NUM_TREES)
regr.fit(train_x, train_y)
print('Time to train: {:.0f} seconds'.format(time.time() - start))
!aplay /home/tom/bicycle_bell.wav

In [ ]:
# Get the predictions
y_preds = regr.predict(test_x)
y_preds = pd.DataFrame(y_preds, columns=['pred'])

In [ ]:
# Compare actual to predicted
actual = pd.DataFrame({'spend_per_hh': pd.Series(test_y)})
actual = actual.reset_index(drop=True)
actual.columns =['actual']

results = y_preds.join(actual)

results['diff'] = abs(results['actual'] - results['pred'])

print('{:<10}{:<10}{}'.format('Column', 'Mean', 'StdDev'))
for x in results.columns:
    print('{:<10}{:<10.2f}{:.2f}'.format(x,results[x].mean(),results[x].std()))

In [ ]:
import matplotlib.mlab as mlab
bins = [-10000, -700,-600,-500,-400,-200,0,200,400,500,600,700,10000]
n, bins, patches = plt.hist(results['pred']-results['actual'], bins=bins, facecolor='green', alpha=.3)
plt.show()

In [ ]:
# This will show the relative importance of each feature
relative_importance = list(zip(COLS, regr.feature_importances_))
print('{:<20}{}'.format('Feature', 'Relative Importance'))
for a,b in sorted(relative_importance, key=itemgetter(1), reverse=True):
    print('{:<20}{:.2}'.format(a, b))

In [ ]:
count,division = np.histogram(results['diff'])
count.shape